# UC1: Feature Engineering - Late Payment Risk

## 0. Setup

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import pandas as pd
import numpy as np
from src.config import SILVER_FILES, GOLD_UC1_FILE, GOLD_UC1_FEATURES, ID_COLS, TARGET
from src.uc1_late_risk.features import build_gold_features

## 1. Load & Validate Raw Data

### 1.1 Load Data

In [3]:
data_dict = {}
for name, filepath in SILVER_FILES.items():
    data_dict[name] = pd.read_csv(filepath)
    print(f"Loaded {name}: {data_dict[name].shape}")

Loaded users: (1000, 7)
Loaded orders: (2953, 8)
Loaded installments: (5238, 9)
Loaded payments: (5084, 9)
Loaded disputes: (147, 7)
Loaded refunds: (95, 6)
Loaded merchants: (120, 7)
Loaded checkout_events: (5474, 5)


### 1.2 Data Quality Validation

In [4]:
def validate_raw_data(data_dict):
    """Validate raw data quality"""
    issues = []
    
    for name, df in data_dict.items():
        # Check for completely empty columns
        empty_cols = df.columns[df.isnull().all()].tolist()
        if empty_cols:
            issues.append(f"{name}: Empty columns {empty_cols}")
        
        # Check for duplicate IDs
        id_col = [c for c in df.columns if c.endswith('_id')][0]
        if df[id_col].duplicated().any():
            issues.append(f"{name}: Duplicate IDs in {id_col}")
        
        # Check critical nulls
        if name == "installments":
            if df["due_date"].isnull().any():
                issues.append(f"{name}: NULL due_dates found")
    
    if issues:
        print("\n".join(issues))
        raise ValueError("Data validation failed!")
    else:
        print("All raw data validation checks passed")

validate_raw_data(data_dict)

All raw data validation checks passed


### 1.3 Data Profiling

In [5]:
for name, df in data_dict.items():
    print(f"\n=== {name} ({df.shape}) ===")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Missing: {df.isnull().sum().sum()} cells")
    print(f"Duplicates: {df.duplicated().sum()} rows")


=== users ((1000, 7)) ===
Columns: ['user_id', 'signup_date', 'kyc_level', 'city', 'account_status', 'created_at', 'updated_at']
Missing: 0 cells
Duplicates: 0 rows

=== orders ((2953, 8)) ===
Columns: ['order_id', 'user_id', 'merchant_id', 'order_date', 'amount', 'currency', 'installments_count', 'status']
Missing: 0 cells
Duplicates: 0 rows

=== installments ((5238, 9)) ===
Columns: ['installment_id', 'order_id', 'user_id', 'merchant_id', 'installment_number', 'due_date', 'paid_date', 'status', 'late_days']
Missing: 828 cells
Duplicates: 0 rows

=== payments ((5084, 9)) ===
Columns: ['payment_id', 'installment_id', 'order_id', 'user_id', 'merchant_id', 'payment_date', 'amount', 'payment_channel', 'status']
Missing: 0 cells
Duplicates: 0 rows

=== disputes ((147, 7)) ===
Columns: ['dispute_id', 'order_id', 'user_id', 'merchant_id', 'dispute_date', 'reason', 'status']
Missing: 0 cells
Duplicates: 0 rows

=== refunds ((95, 6)) ===
Columns: ['refund_id', 'order_id', 'user_id', 'merchant

## 2. Build Gold Features

In [6]:
print("Building features...")
gold_uc1 = build_gold_features(SILVER_FILES)

print(f"Gold features created: {gold_uc1.shape}")
gold_uc1.head()

Building features...
Gold features created: (4824, 49)


,installment_id,order_id,user_id,merchant_id,installment_number,due_date,paid_date,status,late_days,anchor_date,...,checkout_abandon_rate_30d,checkout_friction_score,merchant_name,category,city_merchant,merchant_status,merchant_status_num,merchant_dispute_rate_90d,merchant_refund_rate_90d,merchant_risk_score
0,inst_0000001,order_000006,user_00002,merchant_0109,1,2026-01-31,2026-01-30,paid,0.0,2026-01-31,...,0.5,1.693147,Merchant 109,electronics,Casablanca,active,1,0.000000,0.000000,0.000000
1,inst_0000003,order_000006,user_00002,merchant_0109,3,2026-04-01,2026-04-06,late,5.0,2026-04-01,...,0.0,0.000000,Merchant 109,electronics,Casablanca,active,1,0.000000,0.000000,0.000000
2,inst_0000004,order_000007,user_00002,merchant_0053,1,2025-12-10,2025-12-10,paid,0.0,2025-12-10,...,0.0,0.000000,Merchant 53,travel,Marrakech,active,1,0.000000,0.000000,0.000000
3,inst_0000005,order_000007,user_00002,merchant_0053,2,2026-01-09,2026-01-08,paid,0.0,2026-01-09,...,0.0,0.000000,Merchant 53,travel,Marrakech,active,1,0.130435,0.043478,0.304348
4,inst_0000006,order_000007,user_00002,merchant_0053,3,2026-02-08,2026-02-07,paid,0.0,2026-02-08,...,1.0,2.693147,Merchant 53,travel,Marrakech,active,1,0.166667,0.111111,0.444444


## 3. Feature Validation (CRITICAL!)

### 3.1 Schema Validation

In [7]:
# Check required columns exist
required_cols = set(ID_COLS + [TARGET] + GOLD_UC1_FEATURES)
missing_cols = required_cols - set(gold_uc1.columns)
if missing_cols:
    raise ValueError(f"Missing columns: {missing_cols}")
print("All required columns present")

All required columns present


### 3.2 Data Quality Checks

In [8]:
print("\n=== Data Quality Validation ===")

# Duplicates
dup_rows = gold_uc1.duplicated().sum()
dup_inst = gold_uc1["installment_id"].duplicated().sum()
print(f"Duplicated rows: {dup_rows}, Duplicated IDs: {dup_inst}")
if dup_inst > 0:
    raise ValueError("Duplicate installment IDs found!")

# Target distribution
print(f"\nTarget balance:")
print(gold_uc1[TARGET].value_counts())
print(f"Late rate: {gold_uc1[TARGET].mean():.2%}")

if gold_uc1[TARGET].mean() < 0.01 or gold_uc1[TARGET].mean() > 0.99:
    print("WARNING: Extreme class imbalance detected")

# Check for NaN in target
if gold_uc1[TARGET].isnull().any():
    raise ValueError("NULL values in target!")


=== Data Quality Validation ===
Duplicated rows: 0, Duplicated IDs: 0

Target balance:
is_late
0    3869
1     955
Name: count, dtype: int64
Late rate: 19.80%


### 3.3 Feature Quality Validation

In [9]:
print("\n=== Feature Quality Validation ===")

# Missing data
miss = gold_uc1[GOLD_UC1_FEATURES].isnull().mean().sort_values(ascending=False)
print("\nTop 10 features by missingness:")
print(miss.head(10))

high_missing = miss[miss > 0.8].index.tolist()
if high_missing:
    print(f"WARNING: Features with >80% missing: {high_missing}")

# Constant features (will be removed in training)
const_features = [c for c in GOLD_UC1_FEATURES 
                  if gold_uc1[c].nunique(dropna=False) <= 1]
print(f"\nConstant features (will be auto-removed): {const_features}")

# Check for infinite values
inf_features = [c for c in GOLD_UC1_FEATURES 
                if gold_uc1[c].dtype in ['float64', 'int64'] 
                and np.isinf(gold_uc1[c]).any()]
if inf_features:
    raise ValueError(f"Infinite values in: {inf_features}")


=== Feature Quality Validation ===

Top 10 features by missingness:
sum_order_amount_30d        0.523217
avg_order_amount_30d        0.523217
max_order_amount_30d        0.523217
avg_late_days_90d           0.156716
on_time_payment_rate_90d    0.156716
max_late_days_90d           0.156716
late_payment_rate_90d       0.156716
account_age_days            0.000000
user_city                   0.000000
kyc_level_num               0.000000
dtype: float64

Constant features (will be auto-removed): ['currency', 'merchant_status_num']


### 3.4 Leakage Detection (CRITICAL!)

In [10]:
print("\n=== Leakage Detection ===")

# Features that should NOT be in the dataset
leakage_keywords = ["paid_date", "late_days", "status", "payment_date"]
leakage_cols = [c for c in gold_uc1.columns 
                if any(kw in c.lower() for kw in leakage_keywords)]

# These are OK in the dataset for ID/audit, but NOT in features
leakage_in_features = [c for c in leakage_cols if c in GOLD_UC1_FEATURES]

if leakage_in_features:
    raise ValueError(f"LEAKAGE DETECTED in features: {leakage_in_features}")
else:
    print("No leakage detected in feature list")

print(f"Leakage columns present (but not in features): {leakage_cols}")


=== Leakage Detection ===


ValueError: LEAKAGE DETECTED in features: ['account_status_num', 'avg_late_days_90d', 'max_late_days_90d', 'merchant_status_num']

### 3.5 Temporal Consistency

In [ ]:
# Ensure anchor_date is not null
if gold_uc1["anchor_date"].isnull().any():
    raise ValueError("NULL anchor_dates found!")

# Check date logic
gold_uc1["anchor_date"] = pd.to_datetime(gold_uc1["anchor_date"])
if (gold_uc1["account_age_days"] < 0).any():
    print("WARNING: Negative account ages detected")

### 3.6 Numeric Distribution Checks

In [ ]:
print("\n=== Numeric Distribution Validation ===")

numeric_features = gold_uc1[GOLD_UC1_FEATURES].select_dtypes(include=['number']).columns

# Check for suspicious distributions
for col in numeric_features:
    data = gold_uc1[col].dropna()
    if len(data) > 0:
        q99 = data.quantile(0.99)
        q01 = data.quantile(0.01)
        
        # Check for extreme outliers (>100x difference)
        if q99 > 0 and q01 > 0 and (q99 / q01) > 100:
            print(f"{col}: Extreme range detected (1%={q01:.2f}, 99%={q99:.2f})")

## 4. Generate Validation Report

In [ ]:
validation_report = {
    "dataset_shape": gold_uc1.shape,
    "n_features": len(GOLD_UC1_FEATURES),
    "n_constant_features": len(const_features),
    "target_mean": gold_uc1[TARGET].mean(),
    "duplicates": dup_rows,
    "max_missing_rate": miss.max(),
    "validation_timestamp": pd.Timestamp.now()
}

print("\n=== Validation Report ===")
for k, v in validation_report.items():
    print(f"{k}: {v}")

## 5. Save Gold Dataset

In [ ]:
gold_uc1.to_csv(GOLD_UC1_FILE, index=False)
print(f"\nSaved: {GOLD_UC1_FILE}")
print(f"Shape: {gold_uc1.shape}")
print(f"Features: {len(GOLD_UC1_FEATURES)}")